<a href="https://colab.research.google.com/github/winnie-224/RecommenderSystems/blob/main/BatchRecSyswithSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Batch architecture in Recommendation Systems , most common architecture used in recommendation systems. A batch process that will run periodically to update the recommendations.
The batch process will read the data from the data sources, train a machine learning algorithm, score the model to produce the top k recommendations for every user, and finally, store the recommendations in a dataset.

Once the data is in the database, it can be query from the front end of the website or via an internal backend process. The recommendations will be shown to the user by just doing a SELECT into the database.

In [2]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 747.7/747.7 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.8/919.8 kB 48.8 MB/s eta 0:00:00
     ━━━

In [3]:
import numpy as np
import logging
import sqlite3

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.sar import SAR

In [4]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

# Other data settings
USER_COL = "userID"
ITEM_COL = "itemID"
RATING_COL = "rating"
TIMESTAMP_COL = "timestamp"
PREDICTION_COL = "prediction"

# Model settings
SIMILARITY_TYPE = "jaccard"
TIME_DECAY = 30 # number of days until the weight of the ratings are decayed by 1/2
SEED = 42

# Database parameters
DATABASE = "recodb"
TABLE_NAME = "recommendations"


In [5]:
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(levelname)-8s %(message)s")


In [7]:
# Prepare Data
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE
)

# Convert the float precision to 32-bit in order to reduce memory consumption
data[RATING_COL] = data[RATING_COL].astype(np.float32)

data.head()


100%|██████████| 4.81k/4.81k [00:00<00:00, 16.6kKB/s]


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [8]:
# Score all the data using SAR
model = SAR(
    col_user=USER_COL,
    col_item=ITEM_COL,
    col_rating=RATING_COL,
    col_timestamp=TIMESTAMP_COL,
    similarity_type=SIMILARITY_TYPE,
    time_decay_coefficient=30,
    timedecay_formula=True,
    normalize=True
)

In [9]:
with Timer() as train_time:
    model.fit(data)

print(f"Took {train_time.interval} seconds for training.")

Took 1.3843993590000991 seconds for training.


In [10]:
with Timer() as scoring_time:
    top_k = model.recommend_k_items(data, top_k=TOP_K, remove_seen=True)

print("Took {} seconds for scoring.".format(scoring_time.interval))

Took 0.39142082000012124 seconds for scoring.


In [14]:
top_k.sort_values(by="prediction", ascending=False, inplace=True)

top_k.head(20)

,userID,itemID,prediction
5230,532,69,4.665657
5231,532,172,4.645321
5232,532,423,4.643408
8460,849,204,4.628795
5233,532,174,4.619922
2140,118,195,4.619201
9190,928,174,4.594107
2141,118,183,4.583690
5234,532,385,4.578142
2142,118,89,4.577481


In [15]:
top_k_recommendations = top_k.groupby("userID").head(TOP_K)
top_k_recommendations

,userID,itemID,prediction
5230,532,69,4.665657
5231,532,172,4.645321
5232,532,423,4.643408
8460,849,204,4.628795
5233,532,174,4.619922
...,...,...,...
6775,685,245,1.496370
6776,685,307,1.482737
6777,685,313,1.475710
6778,685,294,1.466042


In [17]:
# Values for user_id = 54
user_id = 54
items_seen = data[data[USER_COL] == user_id]
items_seen

,userID,itemID,rating,timestamp
232,54,106,3.0,880937882
336,54,595,3.0,880937813
512,54,742,5.0,880934806
806,54,302,4.0,880928519
1352,54,676,5.0,880935294
...,...,...,...,...
68542,54,634,1.0,892681013
70980,54,250,4.0,880933834
74116,54,823,2.0,880938088
78663,54,405,4.0,880934806


In [18]:
items_predicted = top_k[top_k[USER_COL] == user_id].sort_values(
    by=PREDICTION_COL, ascending=False
)
items_predicted

,userID,itemID,prediction
1300,54,300,2.784323
1301,54,294,2.601673
1302,54,248,2.548543
1303,54,286,2.458506
1304,54,282,2.436808
1305,54,271,2.433754
1306,54,293,2.368300
1307,54,315,2.367518
1308,54,222,2.357715
1309,54,301,2.354047


In [19]:
# Batch Deployment
# Establish a connection to the database
conn = sqlite3.connect(database=DATABASE)

# Create a cursor object to execute SQL queries
cur = conn.cursor()
# Drop table if it already exists
query = "DROP TABLE IF EXISTS " + TABLE_NAME + ";"
cur.execute(query)
# Create a table to store your data
create_table_query = f"""
CREATE TABLE {TABLE_NAME} (
    user_id INT PRIMARY KEY,
    item1 TEXT,
    item2 TEXT,
    item3 TEXT,
    item4 TEXT,
    item5 TEXT,
    item6 TEXT,
    item7 TEXT,
    item8 TEXT,
    item9 TEXT,
    item10 TEXT
);
"""
cur.execute(create_table_query)

# Commit the changes and close the connection
conn.commit()

In [20]:
# Create a function to prepare and return the data for insertion
def prepare_data(user_group):
    user_id = user_group.name
    recommendations = user_group["itemID"].tolist()
    recommendations.extend([None] * (TOP_K - len(recommendations)) )  # Fill empty slots with NULL
    return (user_id, *recommendations)

# Use apply and groupby to efficiently generate the insert_data list
insert_data = top_k_recommendations.groupby("userID").apply(prepare_data).tolist()

In [21]:
# Define the SQL statement for the bulk insert
insert_sql = f"""
    INSERT INTO {TABLE_NAME} (user_id, item1, item2, item3, item4, item5, item6, item7, item8, item9, item10)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

# Use executemany to insert the data in a single transaction
cur.executemany(insert_sql, insert_data)


# Commit the changes
conn.commit()

In [22]:
query = f"SELECT * FROM {TABLE_NAME} WHERE user_id = {user_id}"
cur.execute(query)
data = cur.fetchall()
print(data)

[(54, '300', '294', '248', '286', '282', '271', '293', '315', '222', '301')]


In [23]:
 # Close the database connection
cur.close()
conn.close()

The batch architecture is the most common and simple architecture in recommendation systems. It works very well when the data is not changing very often, it is easy to implement, and it has a very low latency.

Real examples of industries using this architecture are retail, media and entertainment, ads, gaming and travel.